In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Exploration des fréquentations des gares

In [2]:
frequentations = pd.read_csv("../data/raw/frequentation-gares.csv",sep=";")
print(frequentations.shape)
frequentations.head()

(3010, 22)


,Nom de la gare,Code UIC,Code postal,Segmentation DRG,Total Voyageurs 2023,Total Voyageurs + Non voyageurs 2023,Total Voyageurs 2022,Total Voyageurs + Non voyageurs 2022,Total Voyageurs 2021,Total Voyageurs + Non voyageurs 2021,...,Total Voyageurs 2019,Total Voyageurs + Non voyageurs 2019,Total Voyageurs 2018,Total Voyageurs + Non voyageurs 2018,Total Voyageurs 2017,Total Voyageurs + Non voyageurs 2017,Total Voyageurs 2016,Total Voyageurs + Non voyageurs 2016,Total Voyageurs 2015,Total Voyageurs + Non voyageurs 2015
0,Abbaretz,87481614,44170,C,54862,54862,40825,40825,27466,27466,...,38473,38473,38027,38027,35637,35637,33021,33021,35989,35989
1,Achères Grand Cormier,87386052,78100,B,188426,188426,177092,177092,209673,209673,...,221003,221003,228032,228032,235698,235698,238687,238687,241046,241046
2,Achiet-le-Grand,87342048,62121,C,82597,82597,80648,80648,71432,71432,...,97848,97848,92323,92323,96451,96451,92084,92084,92018,92018
3,Agay,87757559,83530,C,39364,39364,32800,32800,25094,25094,...,22284,22284,15093,15093,14154,14154,19240,19240,19121,19121
4,Aigues-Mortes,87775858,30220,C,17034,17034,11325,11325,8201,8201,...,6146,6146,5311,5311,4822,4822,3546,3546,3917,3917


In [3]:
frequentations.dtypes

Nom de la gare                          object
Code UIC                                 int64
Code postal                              int64
Segmentation DRG                        object
Total Voyageurs 2023                     int64
Total Voyageurs + Non voyageurs 2023     int64
Total Voyageurs 2022                     int64
Total Voyageurs + Non voyageurs 2022     int64
Total Voyageurs 2021                     int64
Total Voyageurs + Non voyageurs 2021     int64
Total Voyageurs 2020                     int64
Total Voyageurs + Non voyageurs 2020     int64
Total Voyageurs 2019                     int64
Total Voyageurs + Non voyageurs 2019     int64
Total Voyageurs 2018                     int64
Total Voyageurs + Non voyageurs 2018     int64
Total Voyageurs 2017                     int64
Total Voyageurs + Non voyageurs 2017     int64
Total Voyageurs 2016                     int64
Total Voyageurs + Non voyageurs 2016     int64
Total Voyageurs 2015                     int64
Total Voyageu

In [4]:
print(frequentations.head())

          Nom de la gare  Code UIC  Code postal Segmentation DRG  \
0               Abbaretz  87481614        44170                C   
1  Achères Grand Cormier  87386052        78100                B   
2        Achiet-le-Grand  87342048        62121                C   
3                   Agay  87757559        83530                C   
4          Aigues-Mortes  87775858        30220                C   

   Total Voyageurs 2023  Total Voyageurs + Non voyageurs 2023  \
0                 54862                                 54862   
1                188426                                188426   
2                 82597                                 82597   
3                 39364                                 39364   
4                 17034                                 17034   

   Total Voyageurs 2022  Total Voyageurs + Non voyageurs 2022  \
0                 40825                                 40825   
1                177092                                177092   
2    

In [5]:
list(frequentations.columns)

['Nom de la gare',
 'Code UIC',
 'Code postal',
 'Segmentation DRG',
 'Total Voyageurs 2023',
 'Total Voyageurs + Non voyageurs 2023',
 'Total Voyageurs 2022',
 'Total Voyageurs + Non voyageurs 2022',
 'Total Voyageurs 2021',
 'Total Voyageurs + Non voyageurs 2021',
 'Total Voyageurs 2020',
 'Total Voyageurs + Non voyageurs 2020',
 'Total Voyageurs 2019',
 'Total Voyageurs + Non voyageurs 2019',
 'Total Voyageurs 2018',
 'Total Voyageurs + Non voyageurs 2018',
 'Total Voyageurs 2017',
 'Total Voyageurs + Non voyageurs 2017',
 'Total Voyageurs 2016',
 'Total Voyageurs + Non voyageurs 2016',
 'Total Voyageurs 2015',
 'Total Voyageurs + Non voyageurs 2015']

## Transformation de la donnée

On remarque que les données sont présentées de la façon suivante:

| Nom                 | Code Postal | Total Voyageurs 2023 | Total Voyageurs 2022 | Total Voyageurs 2021 |
|---------------------|-------------|----------------------|----------------------|----------------------|
| Aiguebelette-le-Lac | 73610       | 4394                 | 4046                 | 3658                 |
| Aigueperse          | 63260       | 86794                | 70599                | 55143                |
| ...                 | ...         | ...                  | ...                  | ...                  |

Ce format n'est pas idéal, car il multiplie le nombre de colonnes si on ajoute des années. Ainsi, il rend compliqué les comparaisons d'une année à une autre.

Il est donc préférable de procéder ainsi : 

| Nom                 | Code Postal | Année | Total Voyageurs      |
|---------------------|-------------|-------|----------------------|
| Aiguebelette-le-Lac | 73610       | 2023  | 4394                 |
| Aiguebelette-le-Lac | 73610       | 2022  | 4046                 |
| Aiguebelette-le-Lac | 73610       | 2020  | 3658                 |
| Aigueperse          | 63260       | 2023  | 86794                |
| Aigueperse          | 63260       | 2022  | 70599                |
| Aigueperse          | 63260       | 2020  | 55143                |
| ...                 | ...         | ...   | ...                  |

On ajoute alors un certain nombre de lignes, mais on rend la donnée beaucoup plus lisible.

In [6]:
def set_correct_columns(frequentations_df):
    years = [str(year) for year in range(2015, 2024)]
    transformed_df = pd.DataFrame()
    for year in years:
        year_df = frequentations_df[["Nom de la gare", "Code UIC", "Code postal", "Segmentation DRG", f"Total Voyageurs {year}", f"Total Voyageurs + Non voyageurs {year}"]]
        year_df = year_df.assign(Année=year) # On peut faire year_df["Année"] = year mais c'est moins propre, on a le warning SettingWithCopyWarning.
        year_df = year_df.rename(columns={f"Total Voyageurs {year}":"Total Voyageurs", f"Total Voyageurs + Non voyageurs {year}":"Total Voyageurs + Non Voyageurs"})
        transformed_df = pd.concat([transformed_df, year_df])
    transformed_df = transformed_df.sort_values(by=["Nom de la gare", "Année"])
    transformed_df = transformed_df.reset_index(drop=True)
    return transformed_df
        
    
frequentations_correct_columns = set_correct_columns(frequentations)


In [7]:
frequentations_correct_columns.head()

,Nom de la gare,Code UIC,Code postal,Segmentation DRG,Total Voyageurs,Total Voyageurs + Non Voyageurs,Année
0,Abancourt,87313759,60220,C,39720,39720,2015
1,Abancourt,87313759,60220,C,41096,41096,2016
2,Abancourt,87313759,60220,C,43760,43760,2017
3,Abancourt,87313759,60220,C,40228,40228,2018
4,Abancourt,87313759,60220,C,42685,42685,2019


In [8]:
frequentations_correct_columns.tail()

,Nom de la gare,Code UIC,Code postal,Segmentation DRG,Total Voyageurs,Total Voyageurs + Non Voyageurs,Année
27085,Évry-Courcouronnes,87681387,91000,B,11258509,11258509,2019
27086,Évry-Courcouronnes,87681387,91000,B,5130572,5130572,2020
27087,Évry-Courcouronnes,87681387,91000,B,8307516,8307516,2021
27088,Évry-Courcouronnes,87681387,91000,B,9299239,9299239,2022
27089,Évry-Courcouronnes,87681387,91000,B,9895865,9895865,2023


## Réduction de la donnée

### ```"Segmentation DRG"```

Cette colonne représente la segmentation des gares provenant du [Document de Références des Gares de Voyageurs (DRG)](https://www.garesetconnexions.sncf/sites/default/files/2022-12/drg_2023_-_version_consultation.pdf)

Chaque gare est attribuée d'une catégorie allant de A à C, ces catégories qualifient l'échelle d'importance d'une gare (National, Régional, Local), et sont fortement liées à leur fréquentation.

[Avis n° 2021-048 du 23 septembre 2021, Autorité des Transports](https://www.autorite-transports.fr/wp-content/uploads/2021/10/avis-2021-048-version-publique-drg-2021.pdf) :

- Les **gares de voyageurs d’intérêt national** (dites gares de catégorie « A »), sont celles dont
la fréquentation par des usagers des services nationaux et internationaux est au moins
égale à 250 000 voyageurs ou dont la fréquentation par ces mêmes usagers correspond à
100 % des voyageurs. Le périmètre de gestion de ces gares correspond à une gare de
voyageurs ou à un ensemble fonctionnel de gares de voyageurs;

- Les **gares de voyageurs d’intérêt régional** (dites gares de catégorie « B ») sont celles qui
ne relèvent pas de la catégorie A et dont la fréquentation totale est au moins égale à
100 000 voyageurs. Le périmètre de gestion de ces gares correspond, dans chaque région,
à l’ensemble des gares de cette catégorie ;

- Les **gares de voyageurs d’intérêt local** (dites gares de catégorie « C ») sont celles qui ne
relèvent d’aucune des catégories précédentes. Le périmètre de gestion de ces gares
correspond, dans chaque région, à l’ensemble des gares de cette catégorie.

In [9]:
frequentations_correct_columns["Segmentation DRG"].value_counts()

Segmentation DRG
C    17361
B     8226
A      900
Name: count, dtype: int64

Cette donnée est intéressante à garder, par exemple pour faire une corrélation entre la fréquentation et la catégorie d'une gare.

### ```Nom de la gare```

On a déjà cette donnée dans ```liste-des-gares.geojson```. Et on utilise plutôt comme clé primaire ```Code UIC```.

### ```Code postal```

Cette donnée n'est pas dans liste-des-gares.geojson, donc on peut la garder.

In [10]:
def process_frequentations(frequentations_df : pd.DataFrame) -> pd.DataFrame:
    """
    Voir notebooks/4_frequentation_gares.ipynb
    Traitement des données de frequentation-gares.csv
    
    La transformation consiste à passer d'un format large, où chaque année est une colonne distincte, 
    à un format long, où chaque année est une ligne distincte. Cela facilite les comparaisons et l'analyse 
    des données sur plusieurs années.
    
    Args:
        frequentations_df (pd.DataFrame): DataFrame contenant les données de frequentation-gares.csv
    Returns:
        pd.DataFrame: Dataframe traitée.
    """
    years = [str(year) for year in range(2015, 2024)]
    frequentations_df_processed = pd.DataFrame()
    for year in years:
        year_df = frequentations_df[["Nom de la gare", "Code UIC", "Code postal", "Segmentation DRG", f"Total Voyageurs {year}", f"Total Voyageurs + Non voyageurs {year}"]]
        year_df = year_df.assign(Année=year) # On peut faire year_df["Année"] = year mais c'est moins propre, on a le warning SettingWithCopyWarning.
        year_df = year_df.rename(columns={f"Total Voyageurs {year}":"Total Voyageurs", f"Total Voyageurs + Non voyageurs {year}":"Total Voyageurs + Non Voyageurs"})
        frequentations_df_processed = pd.concat([frequentations_df_processed, year_df])
    frequentations_df_processed = frequentations_df_processed.sort_values(by=["Nom de la gare", "Année"]) # On trie par nom de gare et année pour avoir un affichage plus lisible
    frequentations_df_processed = frequentations_df_processed.drop(columns=["Nom de la gare"])
    frequentations_df_processed = frequentations_df_processed.reset_index(drop=True)
    return frequentations_df_processed

frequentations_processed = process_frequentations(frequentations)
print(frequentations_processed.shape)
frequentations_processed.head()

(27090, 6)


,Code UIC,Code postal,Segmentation DRG,Total Voyageurs,Total Voyageurs + Non Voyageurs,Année
0,87313759,60220,C,39720,39720,2015
1,87313759,60220,C,41096,41096,2016
2,87313759,60220,C,43760,43760,2017
3,87313759,60220,C,40228,40228,2018
4,87313759,60220,C,42685,42685,2019
